In [70]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/top_reddit_posts.csv')
df.info()
df.head()

In [62]:
# drop the index column
df = df.drop(columns='Unnamed: 0')

# format the column names in snake cases
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

# all posts are not original, the is_original_content column is meaningless and can be dropped
print(df.is_original_content.describe())

# drop columns not needed: author, permalink, url, is_original_content
df = df.drop(columns=['author', 'permalink', 'url', 'is_original_content'])

# convert dtype of text columns to string from object
columns = ['title', 'subreddit', 'flair', 'post_id']
df[columns] = df[columns].astype('string')

# creation time is 10-digit integer corresponds to a unit of second
# convert unix timestamp to datetime64
df['creation_time'] = pd.to_datetime(df['creation_time'], unit='s')

# convert subreddit to categorical data type
df['subreddit'] = df['subreddit'].astype('category')

# re-order the columns
df = df[['post_id', 'subreddit', 'title', 'flair', 'comments', 'score', 'creation_time', 'number_of_comments', 'upvote_ratio']]

df.info()

count      6947
unique        1
top       False
freq       6947
Name: is_original_content, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6947 entries, 0 to 6946
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   post_id             6947 non-null   string        
 1   subreddit           6947 non-null   category      
 2   title               6947 non-null   string        
 3   flair               4034 non-null   string        
 4   comments            6947 non-null   object        
 5   score               6947 non-null   int64         
 6   creation_time       6947 non-null   datetime64[ns]
 7   number_of_comments  6947 non-null   int64         
 8   upvote_ratio        6947 non-null   float64       
dtypes: category(1), datetime64[ns](1), float64(1), int64(2), object(1), string(3)
memory usage: 441.5+ KB


In [63]:
# truncate the data to start from 2023-07-15 to make sure the data can be compressed to under 100MB
df = df.loc[df['creation_time'] >= '2023-07-15']

In [64]:
def eval_comments(x: pd.Series):
    # convert a string of list of dictionaries to list of dictionaries
    comment_list = eval(x)
    
    converted_list = []
    
    # Convert keys to lowercase and replace spaces with underscores
    for d in comment_list:
        new_d = {}
        for key, value in d.items():
            # format keys in snake case
            new_key = key.lower().replace(" ", "_")
            new_d[new_key] = value
        
        converted_list.append(new_d)
        
    return converted_list
    
# convert dtype of comments column from string to a list of dictionaries
df['comments'] = df['comments'].apply(func=eval_comments)

In [65]:
# convert the df from "wide" to long format by exploding the comments column
# Step 1: Explode the `comments` column so each dictionary within the list is in a separate row
df = df.explode('comments', ignore_index=True)

# Step 2: Normalize the dictionary in the `comments` column to extract specific fields
df_comments = pd.json_normalize(df['comments'])

In [66]:
# df_posts has a primary key, post_id
df_posts = df.drop(columns="comments").drop_duplicates().reset_index(drop=True)

df_posts.info()

# df_comments have a primary key, comment_id and a foreign key, post_id
df_comments = pd.concat(
    [
        df["post_id"],
        df_comments[["comment_id", "comment_content", "comment_score", "comment_created_utc"]],
    ],
    axis=1,
)

# Remove the rows with no comment info
df_comments = df_comments.dropna().reset_index(drop=True)

# format types for columns
df_comments[['comment_id', 'comment_content']] = df_comments[['comment_id', 'comment_content']].astype('string')
df_comments['comment_score'] = df_comments['comment_score'].astype('int64')
df_comments['comment_created_utc'] = pd.to_datetime(df_comments['comment_created_utc'], unit='s')

df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4275 entries, 0 to 4274
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   post_id             4275 non-null   string        
 1   subreddit           4275 non-null   category      
 2   title               4275 non-null   string        
 3   flair               2539 non-null   string        
 4   score               4275 non-null   int64         
 5   creation_time       4275 non-null   datetime64[ns]
 6   number_of_comments  4275 non-null   int64         
 7   upvote_ratio        4275 non-null   float64       
dtypes: category(1), datetime64[ns](1), float64(1), int64(2), string(3)
memory usage: 238.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983115 entries, 0 to 983114
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   post_id 

In [67]:
df_posts.head()

,post_id,subreddit,title,flair,score,creation_time,number_of_comments,upvote_ratio
0,1501ibb,AskReddit,"What Worst possible reply to ""I'm pregnant""?",<NA>,11202,2023-07-15 04:06:56,12625,0.85
1,14zz798,AskReddit,What is a personal story you have that you don...,<NA>,6607,2023-07-15 02:12:11,2891,0.97
2,14zx7iq,AskReddit,How did that person in your class become rich?,<NA>,4251,2023-07-15 00:38:28,2542,0.94
3,150aarj,AskReddit,What is the worst reply to “I’m leaving you”?,<NA>,3608,2023-07-15 12:03:43,3564,0.89
4,150j9g1,AskReddit,Who's the Most Dangerous Human Alive Right Now?,<NA>,2287,2023-07-15 18:28:51,3389,0.66


In [68]:
df_comments.head()

,post_id,comment_id,comment_content,comment_score,comment_created_utc
0,1501ibb,js0zukm,Did you get a second opinion?,6894,2023-07-15 04:18:00
1,1501ibb,js1079r,On purpose?,15404,2023-07-15 04:21:36
2,1501ibb,js1269h,"""Why does this keep happening to me?! Second t...",11788,2023-07-15 04:41:47
3,1501ibb,js11llk,“Is it yours?”,24645,2023-07-15 04:35:52
4,1501ibb,js0yusy,Congrats! Whose is it?,8216,2023-07-15 04:07:51


In [69]:
df_posts.to_parquet(path='data/posts.parquet.brotli', compression='brotli', index=False)
df_comments.to_parquet(path='data/comments.parquet.brotli', compression='brotli', index=False)